In [ ]:
import sys
sys.path.append('C:\ProgramData\Anaconda2\Lib')
sys.path.append('C:\python27\lib\site-packages')

import os
import time
import datetime
import io
import pyodbc
import csv
import sys
import struct
import atexit
import mpld3
from mpld3 import plugins
from mpld3.utils import get_id
import collections
import matplotlib.pyplot as plt
from signal import SIGTERM
import numpy as np
import copy

from reportGen import writeReport

from classifier import Classifier
from graphgen import GraphGen
#---------- GLOBAL SETTINGS VARIABLES ----------------------

TIMEINTERVAL = 2
#timeinterval is needed so that the time between
#datetime.now() and the query is not
#infinitesimally small - measured in seconds

REFRESHTIME = 5

BLOBNUMBER = 5
#The number of 4-set sensor blobs allowed
#If BLOBNUMBER = 5, then 20 sensor blobs will be collected

#-----------------------------------------------------------

class SampleRow(object):
    '''Row of Data from Query with Blob Cracking'''
    def __init__(self,segment_id,time,rpm,hexData,crackedData):
        self.segment_id = segment_id
        self.time = time
        self.rpm = rpm
        self.hexData = hexData
        self.crackedData = crackedData
    '''Crack the Hex Data to an array of ints, return into crackedData'''
    def crackHexData(self):
        bytearr = bytearray(self.hexData)
        self.crackedData = []
        i=0
        j=1
        while (j<4096): #4096 bytes makes 2048 16-bit ints
            x1 = hex(bytearr[i])[2:].zfill(2)
            y1 = hex(bytearr[j])[2:].zfill(2)
            v1 = int(str(y1+x1), 16)
            self.crackedData.extend([v1])
            i = i+2
            j = j+2


#def getUTCtime():
    #now = datetime.datetime.now() - datetime.timedelta(seconds=TIMEINTERVAL)
    #utctime = str(now + datetime.timedelta(hours=7))
    #utctime = "2017-05-04 21:16:00" #------------ For Testing
    #return utctime[:-3]

def executeQuery(cursor):
    #qcommand = "select segment_id,gmt,start_rev_rpm,sample_data from dbo.waveform where (segment_id = 218 or segment_id = 244 or segment_id = 270 or segment_id = 296) and gmt > '" + str(utctime) + "' order by gmt, segment_id"
    totalRows = BLOBNUMBER*4
    qcommand = "select top " + str(totalRows) + " segment_id,gmt,start_rev_rpm,sample_data from dbo.waveform where (segment_id = 218 or segment_id = 244 or segment_id = 270 or segment_id = 296) order by gmt desc, segment_id"
    cursor.execute(qcommand)

def checkRowLoopValid(rowCount, classRow):
    if  ((rowCount % 4 == 0 and classRow.segment_id != 218) or
        (rowCount % 4 == 1 and classRow.segment_id != 244) or
        (rowCount % 4 == 2 and classRow.segment_id != 270) or
        (rowCount % 4 == 3 and classRow.segment_id != 296)):
        #print "INVADLID DATA-------------------------------------------- CONSECUTIVE IDs ---------------"
        return False
    return True



class RunForeverBatch(object):
    def __init__(self):
        self.myGen = GraphGen()
        self.run()     
    def run(self):
        testCount = 0
        while(1):
            if (testCount == 0):
                testFile = open('TESTDATA_Normal_SlowRoll.csv', "rb")
                print "TESTDATA_Normal_SlowRoll"
            if (testCount == 1):
                testFile = open('TESTDATA_Normal_RampUp.csv', "rb")
                print "TESTDATA_Normal_RampUp"
            if (testCount == 2):
                testFile = open('TESTDATA_Normal_FastRoll.csv', "rb")
                print "TESTDATA_Normal_FastRoll"
            if (testCount == 3):
                testFile = open('TESTDATA_Normal_RampDown.csv', "rb")
                print "TESTDATA_Normal_RampDown"
            if (testCount == 4):
                testFile = open('TESTDATA_BearingRub_SlowRoll.csv', "rb")
                print "TESTDATA_BearingRub_SlowRoll"
            if (testCount == 5):
                testFile = open('TESTDATA_BearingRub_RampUp.csv', "rb")
                print "TESTDATA_BearingRub_RampUp"
            if (testCount == 6):
                testFile = open('TESTDATA_BearingRub_FastRoll.csv', "rb")
                print "TESTDATA_BearingRub_FastRoll"
            if (testCount == 7):
                testFile = open('TESTDATA_BearingRub_RampDown.csv', "rb")
                print "TESTDATA_BearingRub_RampDown"
            if (testCount == 8):
                testFile = open('TESTDATA_Preload_SlowRoll.csv', "rb")
                print "TESTDATA_Preload_SlowRoll"
            if (testCount == 9):
                testFile = open('TESTDATA_Preload_RampUp.csv', "rb")
                print "TESTDATA_Preload_RampUp"
            if (testCount == 10):
                testFile = open('TESTDATA_Preload_FastRoll.csv', "rb")
                print "TESTDATA_Preload_FastRoll"
            if (testCount == 11):
                testFile = open('TESTDATA_Preload_RampDown.csv', "rb")
                print "TESTDATA_Preload_RampDown"
            if (testCount == 12):
                testFile = open('TESTDATA_Unbalance_SlowRoll.csv', "rb")
                print "TESTDATA_Unbalance_SlowRoll"
            if (testCount == 13):
                testFile = open('TESTDATA_Unbalance_RampUp.csv', "rb")
                print "TESTDATA_Unbalance_RampUp"
            if (testCount == 14):
                testFile = open('TESTDATA_Unbalance_FastRoll.csv', "rb")
                print "TESTDATA_Unbalance_FastRoll"
            if (testCount == 15):
                testFile = open('TESTDATA_Unbalance_RampDown.csv', "rb")
                print "TESTDATA_Unbalance_RampDown"
 
            
            reader = csv.reader(testFile)

            timeNow = datetime.datetime.now()
            timeNextQuery = timeNow + datetime.timedelta(seconds=REFRESHTIME)

            dataPassingList = []

            #FULL information on each sensor blob row
            classRowList = [] #WHAT BRIAN NEEDS

            rowCount = 0
            for row in reader:
                valid = True
                classRow = SampleRow(row[0],row[1],row[2],0,row[3])
                underscoreData = classRow.crackedData
                stringData = underscoreData.split('_')
                classRow.crackedData = [int(x) for x in stringData]
                #if (rowCount == 0):
                    #print classRow.crackedData
                data = np.array(classRow.crackedData)
                #if (rowCount == 0):
                    #print data
                sensorBlob = [data, classRow.segment_id]
                classRowList.extend([sensorBlob])
                rowCount = rowCount + 1
            for p in classRowList:
                dataPassingList.extend([p[0]])

            np_dpl_tmp = np.array(dataPassingList)
            np_dpl = np.reshape( np_dpl_tmp, (4, BLOBNUMBER, 2048) )
            X = np.transpose(np_dpl, (1, 2, 0))

            clf = Classifier()
            clf_data = clf.classify(X, "RandomForest")
            cVec = clf_data["classifications"]
            rVec = {"classification_name": cVec["classification_name"],
                   "slow_roll": cVec["unbalance"],
                   "ramp_up": cVec["preload"],
                   "ramp_down": cVec["bearing_rub"],
                   "fast_roll": cVec["safe"]}
            
            cracked_data = clf_data["cracked_data"]

            
            
            #self.myGen.update(classification["classifications"]["confidence_vec"], classification["classifications"]["confidence_vec"], classification["cracked_data"])
            #print classification["cracked_data"].shape

            #update graph image
            self.myGen.update(cVec,rVec,cracked_data)            
            
            #produce pdf
            writeReport( ["mO.png","mX.png","mY.png"], cVec, rVec,False)            

            
            testFile.close()
            testCount = testCount + 1
            if (testCount > 15):
                testCount = 0
            
            
            #TIMER TO WAIT FOR NEXT CALL
            go = False
            while not go:
                timeNow = datetime.datetime.now()
                if timeNow > timeNextQuery:
                    go = True
                    #print "next query"        

        
        
class RunForeverRealTime(object):
    def __init__(self):
        self.myGen = GraphGen()
        self.run()     
    def run(self):
        
        conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                                "Server=AVANTTH-PC;"
                                "Database=System1_Hist;"
                                "Trusted_Connection=yes;")
        cursor = conn.cursor()
        #if not cursor:
            #print "Cannot Connect to SQL Database - Exiting Program"
            #return
        #else:
            #print "program starting"
        
        while(1):
            timeNow = datetime.datetime.now()
            timeNextQuery = timeNow + datetime.timedelta(seconds=REFRESHTIME)

            '''
            -----------WHAT EVERYONE ELSE NEEDS-------------
            ONLY np arrays of each sensor blob
            ALWAYS will be a multiple of 4
            ALWAYS will be ordered in ascending segment_id's
            1st = sid 218 - Motor Side X-Sensor
            2nd = sid 244 - Motor Side Y-Sensor
            3rd = sid 270 - Outer Side X-Sensor
            4th = sid 296 - Outer Side Y-Sensor
            '''
            dataPassingList = []

            #FULL information on each sensor blob row
            classRowList = [] #WHAT BRIAN NEEDS

            #Gets UTC time from Actual Time for use in SQL Query
            #utctime = getUTCtime()

            #Runs the query
            executeQuery(cursor)

            row = cursor.fetchone()

            rowCount = 0
            valid = True

            while row and rowCount<BLOBNUMBER*4 and valid:
                classRow = SampleRow(row[0],row[1],row[2],row[3],0)
                classRow.crackHexData()
                classRow.time = classRow.time - datetime.timedelta(hours=7)
                data = np.array(classRow.crackedData)
                
                valid = checkRowLoopValid(rowCount, classRow)
                
                sensorBlob = [data, classRow.segment_id]
                classRowList.extend([sensorBlob])
                rowCount = rowCount + 1
                #writer.writerow([rowTup])
                row = cursor.fetchone()
                
            for p in classRowList:
                dataPassingList.extend([p[0]])

            '''
            if rowCount % 4 != 0:
                print "INVADLID DATA-------------------------------------------- NOT GROUPED BY 4 ---------------"
            else:
                for p in classRowList:
                    dataPassingList.extend([p[0]])
                    #print p[0]
            '''

                #first column of each row contains labels for the row
                #last column of each row should contain the normal state
                
            

            
            np_dpl_tmp = np.array(dataPassingList)
            np_dpl = np.reshape( np_dpl_tmp, (4, BLOBNUMBER, 2048) )
            X = np.transpose(np_dpl, (1, 2, 0))
            # print X.shape
            
            '''--------------------------------------------------------------------------------------------------
            CLASSIFIER CODE
            --------------------------------------------------------------------------------------------------'''
            # X: (N_SAMPLES * SAMPLE_SIZE(2048) * N_SENSORS(4)) in a numpy array
            
            clf = Classifier()
            clf_data = clf.classify(X, "RandomForest")
            cVec = clf_data["classification_name"]
            rVec = {"classification_name": cVec["classification_name"],
                   "slow_roll": cVec["unbalance"],
                   "ramp_up": cVec["preload"],
                   "ramp_down": cVec["bearing_rub"],
                   "fast_roll": cVec["safe"]}
            cracked_data = clf_data["cracked_data"]
            # cVec: {"classifier_name": string,
            #        "rub": float,
            #        "preload": float,
            #        "bearing_rub": float
            #        "safe": float}
            
            # rVec: {"classifier_name": string,
            #        "slow_roll": float,
            #        "ramp_up": float,
            #        "ramp_down": float
            #        "fast_roll": float}
            '''--------------------------------------------------------------------------------------------------
            REPORT GENERATION CODE
            --------------------------------------------------------------------------------------------------'''
            # cVec: {"classifier_name": string,
            #        "rub": float,
            #        "preload": float,
            #        "bearing_rub": float
            #        "safe": float}
            
            # rVec: {"classifier_name": string,
            #        "slow_roll": float,
            #        "ramp_up": float,
            #        "ramp_down": float
            #        "fast_roll": float}
            
            #update graph image
            self.myGen.update(cVec,rVec,cracked_data)
            
            #produce pdf
            writeReport( ["mO.png","mX.png","mY.png"], cVec, rVec,False)            
            
            
            #TIMER TO WAIT FOR NEXT CALL
            go = False
            while not go:
                timeNow = datetime.datetime.now()
                if timeNow > timeNextQuery:
                    go = True
                    #print "next query"

        #print "program ending"
        conn.close()



#Run Daemon
#RunForeverRealTime()
RunForeverBatch()


TESTDATA_Normal_SlowRoll
Saving Random Forest Model from rf_s0.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s1.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s2.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s3.pkl
Saving Extra Trees Model



[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


[ 0.    0.05  0.5   0.45]
TESTDATA_Normal_RampUp
Saving Random Forest Model from rf_s0.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s1.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s2.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s3.pkl
Saving Extra Trees Model

[ 0.   0.   0.5  0.5]


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


TESTDATA_Normal_FastRoll
Saving Random Forest Model from rf_s0.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s1.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s2.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s3.pkl
Saving Extra Trees Model

[ 1.  0.  0.  0.]


...

TESTDATA_BearingRub_SlowRoll
Saving Random Forest Model from rf_s0.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s1.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s2.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s3.pkl
Saving Extra Trees Model

[ 0.    0.    0.45  0.55]


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


TESTDATA_BearingRub_RampUp
Saving Random Forest Model from rf_s0.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s1.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s2.pkl
Saving Extra Trees Model
Saving Random Forest Model from rf_s3.pkl
Saving Extra Trees Model


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished



[ 0.   0.   0.5  0.5]
